# matching positive scans to negative scans


I think we were going to do this with the metadata file?


In [1]:
import os
import sys
import shutil
import pandas as pd

In [2]:
path2metadatafile = '/Volumes/My Passport for Mac/negative_scans/batch_0/manifest-1660158222427/unified_metadata.csv'
neg_metadata = pd.read_csv(path2metadatafile)
neg_metadata.head()

,Unnamed: 0,Series UID,Collection,3rd Party Analysis,Data Description URI,Subject ID,Study UID,Study Description,Study Date,Series Description,Manufacturer,Modality,SOP Class Name,SOP Class UID,Number of Images,File Size,File Location,Download Timestamp
0,9,1.2.840.113654.2.55.12956211193113193486393566...,NLST,NaN,NaN,100015,1.2.840.113654.2.55.72042510609262695825058314...,NLST-LSS,01-02-2001,2OPAPHMX8000D3433.212039.00.01.75,Philips,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,30,15.79 MB,./NLST/100015/01-02-2001-NA-NLST-LSS-38059/697...,2022-08-10T12:06:07.26
1,285,1.2.840.113654.2.55.59083523304279024931819345...,NLST,NaN,NaN,100392,1.2.840.113654.2.55.12686920875367267733619883...,NLST-LSS,01-02-1999,0OPAPHMX8000D3493.212055.0nullnull,Philips,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,66,34.74 MB,./NLST/100392/01-02-1999-NA-NLST-LSS-77115/988...,2022-08-16T07:07:58.473
2,284,1.2.840.113654.2.55.32623926315258233999508196...,NLST,NaN,NaN,100392,1.2.840.113654.2.55.20851884748187919223605454...,NLST-LSS,01-02-2000,1OPAPHMX8000D3403.212055.0nullnull,Philips,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,72,37.90 MB,./NLST/100392/01-02-2000-NA-NLST-LSS-52396/869...,2022-08-16T07:07:57.619
3,286,1.2.840.113654.2.55.25607972039193815253923076...,NLST,NaN,NaN,100392,1.2.840.113654.2.55.23602920433504799696444473...,NLST-LSS,01-02-2001,2OPAPHMX8000D3403.212055.0nullnull,Philips,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,85,44.74 MB,./NLST/100392/01-02-2001-NA-NLST-LSS-43590/367...,2022-08-16T07:08:02.538
4,197,1.2.840.113654.2.55.23628971939570524748054233...,NLST,NaN,NaN,100260,1.2.840.113654.2.55.11249086440213545689056573...,NLST-LSS,01-02-2000,1OPATOAQUL4C359.43,TOSHIBA,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,89,46.85 MB,./NLST/100260/01-02-2000-NA-NLST-LSS-07813/1.0...,2022-08-16T06:59:32.392


In [3]:
"""
Reading in the data on the abnormal scans. This is the data that we have f
or all our positive scans. We can use this to do tha metaching
"""
positive_scan_data = pd.read_csv('positive_scan_cohort.csv')
positive_scan_data.head()

,Unnamed: 0,pid,study_yr,sct_ab_desc,sct_slice_num,Unnamed: 5
0,3,100004,0,51,26,NaN
1,6,100004,1,51,22,NaN
2,9,100005,0,51,32,NaN
3,13,100005,1,51,38,NaN
4,18,100005,2,51,38,NaN


In [4]:
for column in neg_metadata.columns:
    print('{}: {}'.format(column, type(neg_metadata.iloc[0,:][column])))

Unnamed: 0: <class 'numpy.int64'>
Series UID: <class 'str'>
Collection: <class 'str'>
3rd Party Analysis: <class 'numpy.float64'>
Data Description URI: <class 'numpy.float64'>
Subject ID: <class 'numpy.int64'>
Study UID: <class 'str'>
Study Description: <class 'str'>
Study Date: <class 'str'>
Series Description: <class 'str'>
Manufacturer: <class 'str'>
Modality: <class 'str'>
SOP Class Name: <class 'str'>
SOP Class UID: <class 'str'>
Number of Images: <class 'numpy.int64'>
File Size: <class 'str'>
File Location: <class 'str'>
Download Timestamp: <class 'str'>


In [5]:
set(neg_metadata['Study Date'])

{'01-02-1999', '01-02-2000', '01-02-2001'}

In [6]:
"""
The directory structure of the dataset is as such:

pid -> years -> scan file -> slices

The goal of the code below is to step through this structure and: 

1. pull the pid
2. pull the year
3. use that information to find the sct_slice_num

"""

def isolate_scan(pid, study_yr, ab_scan_data):
    """
    """
    mask1 = ab_scan_data.pid == pid
    mask2 = ab_scan_data.study_yr == study_yr
    sct_slice_num = int(ab_scan_data[mask1&mask2]['sct_slice_num'].values[0])
    return sct_slice_num

def get_slice_nums(path2pid, ab_scan_data):
    """
    We use the information available to us to get the 
    """
    pid = int(path2pid.split('/')[-1])
    years = get_years(path2pid)
    slice_dict = dict()
    for year in years: 
        print('year when isolating in table: {}'.format(year))
        sct_slice_num = isolate_scan(pid, year, ab_scan_data)
        slice_dict[year] = sct_slice_num
    return slice_dict

def get_years(path2pid):
    lst_of_dirs = [folder for folder in os.listdir(path2pid) if folder[0] != '.']
    if not lst_of_dirs:
        print('we got none')
        return None
    years = []
    for folder in lst_of_dirs:
        #print(folder)
        num = int(folder.split('-')[2])
        print('num: {}'.format(num))
        if num == 1999:
            years.append(0)
        if num == 2000:
            years.append(1)
        if num == 2001:
            years.append(2)
    assert len(years) == len(set(years))
    return years

def get_slice_files(path2pid, yr):
    """
    """
    yr_to_str = {
        0: 1999,
        1: 2000,
        2: 2001
    }
    yr_int = yr_to_str[yr]
    folders = [folder for folder in os.listdir(path2pid) if folder[0] != '.']
    #print(folders)
    yr_lst = [folder for folder in folders if (yr_int == int(folder.split('-')[2]))]
    #print(yr_lst)
    assert (len(yr_lst) == 1)
    year_folder = yr_lst[0]
    path2year = os.path.join(path2pid, year_folder)
    scan_folder = [folder for folder in os.listdir(path2year)]
    assert(len(scan_folder) == 1)
    scan_folder = scan_folder[0]
    path2slices = os.path.join(path2year, scan_folder)
    slices = [ file for file in os.listdir(path2slices) if not ('._' in file) ]
    return slices, path2slices

def get_slice_of_interest(interesting_slice_num, lst_of_slices):
    """
    lst of slices has already been cleaned
    """
    for slc in lst_of_slices:
        slc_depth = int(slc.split('-')[1].split('.dcm')[0])
        if interesting_slice_num == slc_depth:
            return slc
    return None

In [7]:
"""
The code to get a specific pid and year...
"""
year_int_string_map = {
    0: '01-02-1999',
    1: '01-02-2000',
    2: '01-02-2001'
}

def isolate_neg_scan(pid, study_yr, metadata):
    """
    """
    year_string = year_int_string_map[study_yr]
    mask1 = metadata['Subject ID'] == pid
    mask2 = metadata['Study Date'] == year_string
    row = metadata[mask1&mask2]
    assert(row.shape[0] == 1)
    return row

def find_match(sct_slice_depth, metadata):
    """
    not sure how this one will work...
    """
    print(sct_slice_depth)
    for index, row in metadata.iterrows():
        if row['Number of Images'] >= sct_slice_depth:
            metadata.at[index, 'Number of Images'] = -1
            return row['File Location']
    return None

def get_unique_name_for_match(match_path):
    """
    """
    split = match_path.split('/')
    match_pid = split[-3]
    match_year = split[-2].split('-')[2]
    match_filename = '-'.join([match_pid, match_year + '.dcm'])
    return match_filename

In [8]:
isolate_neg_scan(100009, 0, neg_metadata)

,Unnamed: 0,Series UID,Collection,3rd Party Analysis,Data Description URI,Subject ID,Study UID,Study Description,Study Date,Series Description,Manufacturer,Modality,SOP Class Name,SOP Class UID,Number of Images,File Size,File Location,Download Timestamp
393,2,1.2.840.113654.2.55.25266208482312797421685593...,NLST,NaN,NaN,100009,1.2.840.113654.2.55.11986719998729907224236081...,NLST-LSS,01-02-1999,0OPAGELSQXD3802.512070.00.11.5,GE MEDICAL SYSTEMS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,156,82.15 MB,./NLST/100009/01-02-1999-NA-NLST-LSS-12631/2.0...,2022-08-10T12:05:36.796


In [9]:
print(neg_metadata.iloc[0,:]['File Location'][2:])

NLST/100015/01-02-2001-NA-NLST-LSS-38059/6974.000000-2OPAPHMX8000D3433.212039.00.01.75-49420


I think I see how to do this: 

* We are lucky in the sense that the metadata has a column called 'File Location' that we can use to literally match path with path. We just need the precursor to the positivies and negatives. Then just save each path to the dictionary. Then, we can create a new directory and copy them to each path. Oce the copies are done, it does not matter what OSIRIX renames them to. 

In [10]:
path2pids = '/Volumes/My Passport for Mac/positive_scans/batch_0/manifest-1660146378931/NLST'

pids = [file for file in os.listdir(path2pids) if file.isnumeric()]

In [11]:
path_2_neg_scans = '/Volumes/My Passport for Mac/negative_scans/batch_0/manifest-1660158222427/'
path_2_positive_scans = '/Volumes/My Passport for Mac/positive_scans/batch_0/manifest-1660146378931/'

In [12]:
neg_dest_path = '/Volumes/My Passport for Mac/debug_dataset/negative'
pos_dest_path = '/Volumes/My Passport for Mac/debug_dataset/positive'

In [13]:
match_dict = dict()

for pid in pids:
    pid_folder = os.path.join(path2pids, pid)
    print('pid: {}'.format(pid))
    slice_dict = get_slice_nums(pid_folder, positive_scan_data)
    print('slices: {}'.format(slice_dict))
    for year in slice_dict.keys():
        sct_slice_num = slice_dict[year]
        
        slices, path2slices = get_slice_files(pid_folder, year)
        slice_file = get_slice_of_interest(slice_dict[year], slices)
        um = find_match(sct_slice_num, neg_metadata)
        print('um: {}'.format(um))
        if not um:
            continue
            
        neg_match_name = get_unique_name_for_match(um)
        pos_match_name = get_unique_name_for_match(path2slices)
        path2negslice = os.path.join(path_2_neg_scans, um[2:])
        try:
            shutil.copyfile(os.path.join(path2negslice, slice_file), os.path.join(neg_dest_path, neg_match_name))
        except:
            print(path2slices)
            slice_file.replace('0', '')
        shutil.copyfile(os.path.join(path2slices, slice_file), os.path.join(pos_dest_path, pos_match_name))
        if um:
            print(os.path.join(path_2_neg_scans, um[2:]))
            match_dict[path2slices] = os.path.join(path_2_neg_scans, um[2:])
        #print(len(slices))
        #print(path2slices)

pid: 100004
num: 1999
num: 2000
year when isolating in table: 0
year when isolating in table: 1
slices: {0: 26, 1: 22}
26
um: ./NLST/100015/01-02-2001-NA-NLST-LSS-38059/6974.000000-2OPAPHMX8000D3433.212039.00.01.75-49420
/Volumes/My Passport for Mac/positive_scans/batch_0/manifest-1660146378931/NLST/100004/01-02-1999-NA-NLST-LSS-63991/1.000000-0OPAGELSPLUSLUNG4102.512080.00.10.75-94287
/Volumes/My Passport for Mac/negative_scans/batch_0/manifest-1660158222427/NLST/100015/01-02-2001-NA-NLST-LSS-38059/6974.000000-2OPAPHMX8000D3433.212039.00.01.75-49420
22
um: ./NLST/100392/01-02-1999-NA-NLST-LSS-77115/9882.000000-0OPAPHMX8000D3493.212055.0nullnull-87921
/Volumes/My Passport for Mac/positive_scans/batch_0/manifest-1660146378931/NLST/100004/01-02-2000-NA-NLST-LSS-74562/1.000000-1OPAGELSPLUSLUNG4102.512080.00.11.5-36531
/Volumes/My Passport for Mac/negative_scans/batch_0/manifest-1660158222427/NLST/100392/01-02-1999-NA-NLST-LSS-77115/9882.000000-0OPAPHMX8000D3493.212055.0nullnull-87921
pid:

In [21]:
set(neg_metadata['Number of Images'])

{-1,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 188,
 190,
 192,
 195,
 196,
 198,
 199,
 204,
 223,
 226,
 228,
 231,
 232,
 235,
 236,
 237,
 239,
 241,
 243,
 248,
 252,
 257,
 260,
 263,
 268}

In [15]:
positive_scan_data[positive_scan_data.pid == 100430]

,Unnamed: 0,pid,study_yr,sct_ab_desc,sct_slice_num,Unnamed: 5
140,1226,100430,1,51,39,NaN


In [16]:
# I downloaded the wrong folder...

In [17]:
neg_metadata.head()

,Unnamed: 0,Series UID,Collection,3rd Party Analysis,Data Description URI,Subject ID,Study UID,Study Description,Study Date,Series Description,Manufacturer,Modality,SOP Class Name,SOP Class UID,Number of Images,File Size,File Location,Download Timestamp
0,9,1.2.840.113654.2.55.12956211193113193486393566...,NLST,NaN,NaN,100015,1.2.840.113654.2.55.72042510609262695825058314...,NLST-LSS,01-02-2001,2OPAPHMX8000D3433.212039.00.01.75,Philips,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,-1,15.79 MB,./NLST/100015/01-02-2001-NA-NLST-LSS-38059/697...,2022-08-10T12:06:07.26
1,285,1.2.840.113654.2.55.59083523304279024931819345...,NLST,NaN,NaN,100392,1.2.840.113654.2.55.12686920875367267733619883...,NLST-LSS,01-02-1999,0OPAPHMX8000D3493.212055.0nullnull,Philips,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,-1,34.74 MB,./NLST/100392/01-02-1999-NA-NLST-LSS-77115/988...,2022-08-16T07:07:58.473
2,284,1.2.840.113654.2.55.32623926315258233999508196...,NLST,NaN,NaN,100392,1.2.840.113654.2.55.20851884748187919223605454...,NLST-LSS,01-02-2000,1OPAPHMX8000D3403.212055.0nullnull,Philips,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,-1,37.90 MB,./NLST/100392/01-02-2000-NA-NLST-LSS-52396/869...,2022-08-16T07:07:57.619
3,286,1.2.840.113654.2.55.25607972039193815253923076...,NLST,NaN,NaN,100392,1.2.840.113654.2.55.23602920433504799696444473...,NLST-LSS,01-02-2001,2OPAPHMX8000D3403.212055.0nullnull,Philips,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,-1,44.74 MB,./NLST/100392/01-02-2001-NA-NLST-LSS-43590/367...,2022-08-16T07:08:02.538
4,197,1.2.840.113654.2.55.23628971939570524748054233...,NLST,NaN,NaN,100260,1.2.840.113654.2.55.11249086440213545689056573...,NLST-LSS,01-02-2000,1OPATOAQUL4C359.43,TOSHIBA,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,-1,46.85 MB,./NLST/100260/01-02-2000-NA-NLST-LSS-07813/1.0...,2022-08-16T06:59:32.392


In [18]:
print(path2slices.split('/')[-3])

101321


In [19]:
print(path2slices.split('/')[-2].split('-')[2])

2001


In [20]:
len(match_dict)

477